In [2]:
pockemon_card = {
    "name": "Pikachu",
    "type": "Electric",
    "HP": 100,
}

schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "type": {"type": "string"},
        "HP": {"type": "integer"},
    },
    "required": ["name", "type", "hp"]
}

exapmle = """
{
    "name": "Pikachu",
    "type": "Electric",
    "HP": 100,
}
 """

schema_string = """ {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "type": {"type": "string"},
        "HP": {"type": "integer"},
    },
    "required": ["name", "type", "hp"]
} """


In [8]:
from openai import OpenAI
from google.colab import userdata

client= OpenAI(
    api_key=userdata.get('TOGETHER_API_KEY_NEW'),
    base_url='https://api.together.xyz/v1'
)

completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    messages=[
        {"role": "user",
         "content": "Generate a pokemon card. use JSON, and awlwys respones in this format: "}],
  response_format={"type": "json_object", "schema": schema}

)

print(completion.choices[0].message.content)

{"name": "Embermaw", "type": "Fire/Dragon", "HP": 120}


In [9]:
tools = [{
    "type": "function",
    "function": {
        "name": "calculator",
        "description": "Useful for when you need to answer questions about math. Use SymPy expresions, eg 2+2",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": { "type": "string" }
            }
        }
    }
}, {
    "type": "function",
    "function": {
        "name": "check_weather",
        "description": "Useful for when you need to check the weather!",
        "parameters": {
            "type": "object",
            "properties": {
                "city_name": { "type": "string", "description": "The name of a city NOT a country" }
            }
        }
    }
}]

completion = client.chat.completions.create(
  model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
  messages=[
    {
        "role": "user",
        "content": "What's the weather in France today?"
    }
  ],
  tools=tools
)


In [10]:
print(completion.choices[0].message.tool_calls[0].function.arguments)
print(completion.choices[0].message.tool_calls[0].function.name)

{"city_name":"Paris"}
check_weather


In [12]:
import requests

def get_weather(city):
  api_key = userdata.get('WEATHER_API_KEY')
  url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}&aqi=no"
  response = requests.get(url)
  weather_data = response.json()
  return weather_data

# Example usage
city = "Karachi"
weather_data = get_weather(city)
print(weather_data)

{'location': {'name': 'Karachi', 'region': 'Sindh', 'country': 'Pakistan', 'lat': 24.8667, 'lon': 67.05, 'tz_id': 'Asia/Karachi', 'localtime_epoch': 1738261981, 'localtime': '2025-01-30 23:33'}, 'current': {'last_updated_epoch': 1738261800, 'last_updated': '2025-01-30 23:30', 'temp_c': 18.3, 'temp_f': 64.9, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 8.9, 'wind_kph': 14.4, 'wind_degree': 282, 'wind_dir': 'WNW', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 45, 'cloud': 0, 'feelslike_c': 18.3, 'feelslike_f': 64.9, 'windchill_c': 20.5, 'windchill_f': 68.9, 'heatindex_c': 20.5, 'heatindex_f': 68.9, 'dewpoint_c': 12.4, 'dewpoint_f': 54.3, 'vis_km': 5.0, 'vis_miles': 3.0, 'uv': 0.0, 'gust_mph': 15.2, 'gust_kph': 24.5}}


In [13]:
import json

chat_history=[
    {
        "role": "user",
        "content": "What's the weather in France today?"
    }
  ]

while True:

  completion = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=chat_history,
    tools=tools
  )

  tool_calls = completion.choices[0].message.tool_calls

  if tool_calls:
    arguments = tool_calls[0].function.arguments
    print(arguments)
    name = tool_calls[0].function.name
    print(name)

    chat_history.append(completion.choices[0].message)

    if name == "check_weather":
      city = json.loads(arguments)["city_name"]
      weather_data = get_weather(city)
      chat_history.append({
          "role": "function",
          "name": name,
          "content": json.dumps(weather_data)
      })
  else:
    print(completion.choices[0].message.content)
    break

{"city_name":"Paris"}
check_weather
The weather in Paris, France today is partly cloudy with a temperature of 6.3°C (43.3°F) and a feels-like temperature of 3.7°C (38.6°F). The wind is blowing at 8.1 mph (13.0 kph) from the north-northeast, and the humidity is 93%. There is no precipitation, and the visibility is 10 km (6 miles). The UV index is 0.0, and the gust speed is 10.6 mph (17.0 kph).
